In [ ]:
"""
一维卷积神经网络用于时间序列预测
================================================

本示例使用一维卷积神经网络预测Jena气候数据集中的温度变化。
一维卷积可以有效捕捉时间序列中的局部模式，相比RNN训练速度更快。

网络架构：
1. Conv1D层：提取时间序列的局部模式
2. MaxPooling层：降采样，减少计算量
3. 多层卷积堆叠：提取多尺度特征
4. GlobalMaxPooling：全局特征提取
5. Dense层：输出温度预测值

数据集：Jena气候数据集（2009-2016年的气候观测数据）
任务：基于过去240小时（10天）的数据预测24小时后的温度
"""

import os
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

# 设置随机种子以保证结果可复现
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# ==================== 数据加载与预处理 ====================

data_dir = "/home/dingziming/下载/jena_climate"
fname = os.path.join(data_dir, "jena_climate_2009_2016.csv")

print("正在加载Jena气候数据集...")
with open(fname) as f:
    data = f.read()

# 解析CSV数据
lines = data.split("\n")
header = lines[0].split(",")
lines = lines[1:]

print(f"数据特征: {header}")
print(f"数据总行数: {len(lines)}")

# 将数据转换为浮点数数组
float_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(",")[1:]]
    float_data[i, :] = values

print(f"数据形状: {float_data.shape}")

# ==================== 数据可视化 ====================

# 提取温度数据（第2列，索引为1）
temp = float_data[:, 1]

# 绘制完整温度时间序列
plt.figure(figsize=(14, 5))
plt.plot(range(len(temp)), temp, linewidth=0.5)
plt.title('Jena气候数据集 - 完整温度时间序列 (2009-2016)', fontsize=14, fontweight='bold')
plt.xlabel('时间步 (10分钟/步)', fontsize=12)
plt.ylabel('温度 (°C)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 绘制前10天的温度数据（1440个时间步，每10分钟一个）
plt.figure(figsize=(14, 5))
plt.plot(range(1440), temp[:1440], linewidth=1.5)
plt.title('Jena气候数据集 - 前10天温度变化', fontsize=14, fontweight='bold')
plt.xlabel('时间步 (10分钟/步)', fontsize=12)
plt.ylabel('温度 (°C)', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# ==================== 数据标准化 ====================

# 使用前200000个样本的统计量进行标准化（避免数据泄露）
mean = float_data[:200000].mean(axis=0)
std = float_data[:200000].std(axis=0)
float_data = (float_data - mean) / std

print(f"数据标准化完成 - 均值: {mean[1]:.2f}°C, 标准差: {std[1]:.2f}°C")

# ==================== 数据生成器 ====================

def generator(data, lookback, delay, min_index, max_index, 
              shuffle=False, batch_size=128, step=6):
    """
    时间序列数据生成器
    
    参数说明：
    - data: 原始时间序列数据 (标准化后)
    - lookback: 输入序列长度（回看多少个时间步）
    - delay: 预测目标与当前时刻的时间间隔
    - min_index, max_index: 数据索引范围
    - shuffle: 是否随机打乱数据
    - batch_size: 批次大小
    - step: 采样步长（用于降采样，节省内存和计算）
    
    返回：
    - samples: 输入序列 shape=(batch_size, lookback//step, features)
    - targets: 目标值 shape=(batch_size,)
    """
    if max_index is None:
        max_index = len(data) - delay - 1
    
    i = min_index + lookback
    
    while True:
        if shuffle:
            # 随机采样：训练集使用
            rows = np.random.randint(min_index + lookback, max_index, size=batch_size)
        else:
            # 顺序采样：验证集和测试集使用
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)
        
        # 初始化样本和目标数组
        samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
        targets = np.zeros((len(rows),))
        
        # 填充数据
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]  # 预测温度（索引1）
        
        yield samples, targets

# ==================== 生成器参数配置 ====================

# 超参数设置
lookback = 1440      # 回看1440步 = 240小时 = 10天 (每步10分钟)
delay = 144          # 预测144步后 = 24小时后的温度
step = 6             # 每隔6步采样一次 = 每小时采样一次
batch_size = 128     # 批次大小

print("\n时间序列参数:")
print(f"- 输入窗口: {lookback}步 = {lookback * 10 / 60:.1f}小时 = {lookback * 10 / 60 / 24:.1f}天")
print(f"- 预测延迟: {delay}步 = {delay * 10 / 60:.1f}小时")
print(f"- 采样步长: {step}步 = {step * 10}分钟")
print(f"- 实际输入序列长度: {lookback // step}步")

# 创建数据生成器
train_gen = generator(
    float_data, lookback=lookback, delay=delay,
    min_index=0, max_index=200000, shuffle=True,
    step=step, batch_size=batch_size
)

val_gen = generator(
    float_data, lookback=lookback, delay=delay,
    min_index=200001, max_index=300000,
    step=step, batch_size=batch_size, shuffle=False
)

test_gen = generator(
    float_data, lookback=lookback, delay=delay,
    min_index=300001, max_index=None,
    step=step, batch_size=batch_size, shuffle=False
)

# 计算验证和测试步数
val_steps = (300000 - 200001 - lookback) // batch_size
test_steps = (len(float_data) - 300001 - lookback) // batch_size

print(f"\n数据集划分:")
print(f"- 训练集: 0 - 200000")
print(f"- 验证集: 200001 - 300000 ({val_steps}步)")
print(f"- 测试集: 300001 - {len(float_data)} ({test_steps}步)")

# ==================== 构建一维卷积神经网络 ====================

model = Sequential([
    # 第一层卷积：32个卷积核，窗口大小5
    # input_shape=(None, features)表示可接受任意长度的序列
    layers.Conv1D(filters=32, kernel_size=5, activation='relu',
                  input_shape=(None, float_data.shape[-1])),
    
    # 第一层池化：降采样，减少计算量
    layers.MaxPooling1D(pool_size=3),
    
    # 第二层卷积：继续提取更复杂的模式
    layers.Conv1D(filters=32, kernel_size=5, activation='relu'),
    
    # 第二层池化
    layers.MaxPooling1D(pool_size=3),
    
    # 第三层卷积：进一步提取高层次特征
    layers.Conv1D(filters=32, kernel_size=5, activation='relu'),
    
    # 全局最大池化：从每个特征图中提取最显著的特征
    layers.GlobalMaxPooling1D(),
    
    # 输出层：回归任务，输出单个数值（温度）
    layers.Dense(1)
])

# 编译模型
model.compile(
    optimizer=RMSprop(learning_rate=1e-3),
    loss='mae',           # 平均绝对误差，适合回归任务
    metrics=['mae']       # 监控指标
)

# 显示模型架构
print("\n模型架构:")
model.summary()

# ==================== 模型训练 ====================

print("\n开始训练模型...")
history = model.fit(
    train_gen,
    steps_per_epoch=500,
    epochs=20,
    validation_data=val_gen,
    validation_steps=val_steps,
    verbose=1
)

# ==================== 可视化训练过程 ====================

# 提取训练历史
loss = history.history['loss']
val_loss = history.history['val_loss']
mae = history.history['mae']
val_mae = history.history['val_mae']
epochs_range = range(1, len(loss) + 1)

# 创建图表
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 绘制损失曲线
ax1.plot(epochs_range, loss, 'bo-', label='训练损失', linewidth=2)
ax1.plot(epochs_range, val_loss, 'ro-', label='验证损失', linewidth=2)
ax1.set_title('训练和验证损失曲线 (MAE)', fontsize=14, fontweight='bold')
ax1.set_xlabel('轮次', fontsize=12)
ax1.set_ylabel('平均绝对误差', fontsize=12)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# 绘制MAE曲线
ax2.plot(epochs_range, mae, 'bo-', label='训练MAE', linewidth=2)
ax2.plot(epochs_range, val_mae, 'ro-', label='验证MAE', linewidth=2)
ax2.set_title('训练和验证MAE曲线', fontsize=14, fontweight='bold')
ax2.set_xlabel('轮次', fontsize=12)
ax2.set_ylabel('平均绝对误差', fontsize=12)
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ==================== 模型评估 ====================

print("\n在测试集上评估模型...")
test_loss = model.evaluate(test_gen, steps=test_steps, verbose=1)

# 将标准化后的MAE转换回实际温度单位
actual_mae = test_loss * std[1]

print("\n" + "="*60)
print("模型性能分析:")
print("="*60)
print(f"最终训练MAE (标准化): {mae[-1]:.4f}")
print(f"最终验证MAE (标准化): {val_mae[-1]:.4f}")
print(f"测试MAE (标准化): {test_loss:.4f}")
print(f"测试MAE (实际温度): {actual_mae:.2f}°C")
print("="*60)

print("\n一维卷积网络特点:")
print("- 训练速度快：相比RNN，卷积操作可以完全并行化")
print("- 参数少：共享卷积核权重，模型更轻量")
print("- 适合局部模式：擅长捕捉时间序列的局部相关性")
print("- 感受野有限：对长期依赖的建模能力弱于RNN")